题目2

In [1]:
!pip install paddlex

Looking in indexes: https://mirror.baidu.com/pypi/simple/
     |████████████████████████████████| 522kB 15.4MB/s eta 0:00:01
     |████████████████████████████████| 1.0MB 13.5MB/s eta 0:00:01
     |████████████████████████████████| 102kB 13.5MB/s ta 0:00:01
     |████████████████████████████████| 153kB 23.7MB/s eta 0:00:01
  Created wheel for pycocotools: filename=pycocotools-2.0.2-cp37-cp37m-linux_x86_64.whl size=278359 sha256=42a97d390026c8d4b6c2680fb737c7ed1c5c0507ced4080a414323f89b798984
  Stored in directory: /home/aistudio/.cache/pip/wheels/fb/44/67/8baa69040569b1edbd7776ec6f82c387663e724908aaa60963
Successfully built pycocotools


In [2]:
!unzip -oq /home/aistudio/data/data81942/00.zip

In [3]:
import paddlex as pdx

In [4]:
#数据集格式转换
!paddlex --data_conversion --source labelme --to MSCOCO \
        --pics 00 \
        --annotations 00 \
        --save_dir dataset_coco

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/setuptools/depends.py:2: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp


In [5]:
#数据集划分
!paddlex --split_dataset --format COCO --dataset_dir dataset_coco --val_value 0.2 --test_value 0.1

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/setuptools/depends.py:2: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  import imp
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/__init__.py:107: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import MutableMapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/rcsetup.py:20: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable, Mapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/colors.py:53: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is depre

In [6]:
from paddlex.det import transforms

# 定义训练和验证时的transforms
train_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(), transforms.Normalize(),
    transforms.ResizeByShort(
        short_size=800, max_size=1333), transforms.Padding(coarsest_stride=32)
])

eval_transforms = transforms.Compose([
    transforms.Normalize(), transforms.ResizeByShort(
        short_size=800, max_size=1333), transforms.Padding(coarsest_stride=32)
])

In [7]:
# 定义训练和验证所用数据集
train_dataset = pdx.datasets.CocoDetection(
    data_dir='dataset_coco/JPEGImages',
    ann_file='dataset_coco/train.json',
    transforms=train_transforms,
    shuffle=True)
eval_dataset = pdx.datasets.CocoDetection(
    data_dir='dataset_coco/JPEGImages',
    ann_file='dataset_coco/val.json',
    transforms=eval_transforms)

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/__init__.py:107: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import MutableMapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/rcsetup.py:20: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable, Mapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/colors.py:53: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sized


loading annotations into memory...
Done (t=0.01s)
creating index...
index created!
2021-04-20 17:12:13 [INFO]	Starting to read file list from dataset...
2021-04-20 17:12:13 [INFO]	293 samples in file dataset_coco/train.json
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!
2021-04-20 17:12:13 [INFO]	Starting to read file list from dataset...
2021-04-20 17:12:13 [INFO]	83 samples in file dataset_coco/val.json


In [8]:
# 初始化模型并训练
num_classes = len(train_dataset.labels) + 1

model = pdx.det.MaskRCNN(num_classes=num_classes, backbone='ResNet50')

model.train(
    num_epochs=100,
    train_dataset=train_dataset,
    train_batch_size=4,
    eval_dataset=eval_dataset,
    learning_rate=0.00125,
    warmup_steps=10,
    lr_decay_epochs=[8, 11],
    save_dir='output/mask_rcnn_r50_fpn',
    use_vdl=True)

/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/math_op_patch.py:298: UserWarning: /opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddlex/cv/nets/detection/fpn.py:172
The behavior of expression A + B has been unified with elementwise_add(X, Y, axis=-1) from Paddle 2.0. If your code works well in the older versions but crashes in this version, try to use elementwise_add(X, Y, axis=0) instead of A + B. This transitional warning will be dropped in the future.
  op_type, op_type, EXPRESSION_MAP[method_name]))
2021-04-20 17:12:34,874 - INFO - If regularizer of a Parameter has been set by 'fluid.ParamAttr' or 'fluid.WeightNormParamAttr' already. The Regularization[L2Decay, regularization_coeff=0.000100] in Optimizer will not take effect, and it will only be applied to other Parameters!
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/fluid/layers/math_op_patch.py:298: UserWarning: /opt/conda/envs/python35-pad

2021-04-20 17:12:35 [INFO]	Downloading ResNet50_cos_pretrained.tar from https://paddle-imagenet-models-name.bj.bcebos.com/ResNet50_cos_pretrained.tar


100%|██████████| 100310/100310 [00:03<00:00, 25577.76KB/s]


2021-04-20 17:12:39 [INFO]	Decompressing output/mask_rcnn_r50_fpn/pretrain/ResNet50_cos_pretrained.tar...
2021-04-20 17:12:45 [INFO]	Load pretrain weights from output/mask_rcnn_r50_fpn/pretrain/ResNet50_cos_pretrained.
2021-04-20 17:12:45 [INFO]	There are 265 varaibles in output/mask_rcnn_r50_fpn/pretrain/ResNet50_cos_pretrained are loaded.
2021-04-20 17:12:53 [INFO]	[TRAIN] Epoch=1/100, Step=2/73, loss=2.656261, loss_cls=0.788994, loss_bbox=0.0, loss_mask=0.950104, loss_rpn_cls=0.696596, loss_rpn_bbox=0.220567, lr=0.0005, time_each_step=3.93s, eta=10:15:8
2021-04-20 17:12:55 [INFO]	[TRAIN] Epoch=1/100, Step=4/73, loss=2.070028, loss_cls=0.468958, loss_bbox=0.0, loss_mask=0.694026, loss_rpn_cls=0.694214, loss_rpn_bbox=0.212831, lr=0.000667, time_each_step=2.34s, eta=6:5:55
2021-04-20 17:12:57 [INFO]	[TRAIN] Epoch=1/100, Step=6/73, loss=1.77785, loss_cls=0.250588, loss_bbox=0.0, loss_mask=0.521612, loss_rpn_cls=0.691239, loss_rpn_bbox=0.314411, lr=0.000833, time_each_step=1.82s, eta=4:4

100%|██████████| 83/83 [00:18<00:00,  4.51it/s]


...

creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.06s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.352
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.833
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.217
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.360
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.366
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.530
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.530
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.

100%|██████████| 83/83 [00:16<00:00,  5.09it/s]


creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.05s).
Accumulating evaluation results...
DONE (t=0.02s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.348
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.813
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.254
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.356
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.363
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.547
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.547
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.

100%|██████████| 83/83 [00:16<00:00,  5.05it/s]


creating index...
index created!
Running per image evaluation...
Evaluate annotation type *bbox*
DONE (t=0.05s).
Accumulating evaluation results...
DONE (t=0.01s).
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.350
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.809
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.223
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.358
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.363
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.535
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.535
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.

In [9]:
model = pdx.load_model('output/mask_rcnn_r50_fpn/best_model')
image_name = 'dataset_coco/JPEGImages/lhgd227.jpg'
result = model.predict(image_name)
pdx.det.visualize(image_name, result, threshold=0.5)

2021-04-20 18:57:38 [INFO]	Model[MaskRCNN] loaded.


/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/cbook/__init__.py:2349: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  if isinstance(obj, collections.Iterator):


2021-04-20 18:57:38 [INFO]	The visualized result is saved as ./visualize_lhgd227.jpg


![](https://ai-studio-static-online.cdn.bcebos.com/8cc39c16ec3949f698b821682e90361c3d3d0f25557d4be3b3383f87ec2e5812)